# NFL EDA

# import libraries

In [ ]:

# general
import os
import gc
import pickle
import glob
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import cv2
import matplotlib.pyplot as plt
import time
import math

# deep learning
import timm
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# loss metrics
from sklearn.metrics import matthews_corrcoef
 
# exp manager
import mlflow

# from torchmetrics.classification import BinaryMatthewsCorrCoef
# matthews_corrcoef = BinaryMatthewsCorrCoef()

# from torchmetrics import MatthewsCorrCoef
# matthews_corrcoef = MatthewsCorrCoef(num_classes=1)#default threshold=0.5

# warningの表示方法の設定
import warnings
warnings.filterwarnings("ignore")


# Set Configurations

In [ ]:
kaggle = False
DEBUG = True
class CFG:
    if kaggle:
        BASE_DIR = "/kaggle/input/nfl-player-contact-detection"
    else:
        BASE_DIR = "/workspace/input"
    TRAIN_HELMET_CSV = os.path.join(BASE_DIR, "train_baseline_helmets.csv")
    TRAIN_TRACKING_CSV = os.path.join(BASE_DIR, "train_player_tracking.csv")
    TRAIN_VIDEO_META_CSV = os.path.join(BASE_DIR, "train_video_metadata.csv")
    TRAIN_LABEL_CSV = os.path.join(BASE_DIR, "train_labels.csv")

    # data config    
    img_size = (224, 224)
    batch_size = 64
    num_workers = 0
    n_fold = 1

    # model config
    model_name = "tf_efficientnet_b0"
    out_features = 1
    inp_channels= 3
    pretrained = True
    model_dir = os.path.join(os.path.dirname(BASE_DIR), "model")
    
    # learning config
    n_epoch = 20
    lr = 1e-6
    T_max = 10
    min_lr = 1e-7
    weight_decay = 1e-6
    
    # etc
    print_freq = 100
    random_seed = 21
    
    MLFLOW_CATEGORY = "make_baseline"
    EXP_NAME = "DEBUG"
    if DEBUG:
        n_epoch = 10
        # epoch_step_valid = 0
        # steps_per_epoch = 10
        

# Utils

In [ ]:
def seed_everything(seed=CFG.random_seed):
    #os.environ['PYTHONSEED'] = str(seed)
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False
seed_everything()

# device optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using device: {device}')

In [ ]:
def asMinutes(s):
    """Convert Seconds to Minutes."""
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    """Accessing and Converting Time Data."""
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

class AverageMeter(object):
    """Computes and stores the average and current value."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Dataset Utils

In [ ]:
def get_snap_frame(row):
    elaped_time_start2snap = row.snap_time - row.start_time
    elaped_seconds = elaped_time_start2snap.seconds
    snap_frame = elaped_seconds*59.95
    return snap_frame

In [ ]:
def set_inimg_window(crop_area, img_size=(720, 1280)):
    left, top, right, bot = crop_area
    # set crop area (area size in img size)
    crop_left = max(0, left)
    crop_top = max(0, top)
    if crop_left != left:   right = right - left
    if crop_top != top:    bot = bot - top
    crop_bot = min(img_size[0], bot)
    crop_right = min(img_size[1], right)
    if crop_bot != bot:    crop_top = crop_top - (bot - img_size[0])
    if crop_right != right:   crop_left = left - (right - img_size[1])
    
    return [crop_left, crop_top, crop_right, crop_bot]

In [ ]:
def make_player_mask(img, helmet_pos, img_size=(720, 1280, 3), alpha=0.3):
    crop_size=[-helmet_pos[2], -helmet_pos[3], helmet_pos[2]*3, helmet_pos[3]*6] # helmetの大きさによってplayerの範囲も変更
    base_area = np.array(helmet_pos) + np.array(crop_size) # [left, top, width, height]
    # set players area
    palyer_area = [base_area[0],  base_area[1], base_area[0] + base_area[2], base_area[1] + base_area[3]]
    palyer_area = set_inimg_window(palyer_area)
    mask = np.zeros(img_size, dtype=np.uint8)
    cv2.rectangle(mask, [palyer_area[0], palyer_area[1]], [palyer_area[2], palyer_area[3]], (255, 255, 255), -1)
    mask = np.clip(mask, 0, 1).astype(np.uint8)
    return mask

In [ ]:
def get_1player_croparea(helmet_pos, img_size=(720, 1280)):
    crop_size=[-helmet_pos[2]*3, -helmet_pos[3]*4, helmet_pos[2]*6, helmet_pos[3]*6] # helmetの大きさによってplayerの範囲も変更
    players_area = np.array(helmet_pos) + np.array(crop_size) # [left, top, width, height]
    # set players area
    crop_area = [players_area[0],  players_area[1], players_area[0] + players_area[2], players_area[1] + players_area[3]]
    crop_area = set_inimg_window(crop_area)
    return crop_area

In [ ]:
def get_2player_croparea(helmet1_pos, helmet2_pos, img_size=(720, 1280)):
    player1_crop_size=[-10, -10, helmet1_pos[2]*4, helmet1_pos[3]*4] # helmetの大きさによってplayerの範囲も変更
    player2_crop_size=[-10, -10, helmet2_pos[2]*4, helmet2_pos[3]*4] # helmetの大きさによってplayerの範囲も変更
    player1_area = np.array(helmet1_pos) + np.array(player1_crop_size) # [left, top, width, height]
    player2_area = np.array(helmet2_pos) + np.array(player2_crop_size) # [left, top, width, height]
    # [left, top, width, height] => [left, top, right, bot]
    player1_crop_area = np.array([player1_area[0], player1_area[1], player1_area[0]+player1_area[2], player1_area[1]+player1_area[3]]) # [left, top, right, bot]
    player2_crop_area = np.array([player2_area[0], player2_area[1], player2_area[0]+player2_area[2], player2_area[1]+player2_area[3]]) # [left, top, right, bot]
    # get min max for set pos in img
    area_min = np.min(np.array([player1_crop_area, player2_crop_area]), axis=0)
    area_max = np.max(np.array([player1_crop_area, player2_crop_area]), axis=0)
    crop_area = [area_min[0], area_min[1], area_max[2], area_max[3]]
    crop_area = set_inimg_window(crop_area)
    return crop_area

In [ ]:
def mask_blend(img, mask):
    img = np.clip(img, 0, 255).astype(np.uint8)
    mask = np.clip(mask, 0, 1).astype(np.uint8)
    img = img*mask
    img = np.clip(img, 0, 255).astype(np.uint8)
    return img

In [ ]:
def search_helmet(helmet_df, player, frame):
    for diff_frame in range(5):
        read_frame = frame + diff_frame
        player_helmet = helmet_df.query('nfl_player_id==@player and frame==@read_frame')
        if len(player_helmet) > 0:
            return player_helmet
        read_frame = frame - diff_frame
        player_helmet = helmet_df.query('nfl_player_id==@player and frame==@read_frame')
        if len(player_helmet) > 0:
            return player_helmet
    return []

# Read Data

In [ ]:
helmet_df = pd.read_csv(CFG.TRAIN_HELMET_CSV)
tracking_df = pd.read_csv(CFG.TRAIN_TRACKING_CSV)
videometa_df = pd.read_csv(CFG.TRAIN_VIDEO_META_CSV, parse_dates=["start_time", "end_time", "snap_time"])
target_df = pd.read_csv(CFG.TRAIN_LABEL_CSV, parse_dates=["datetime"])

In [ ]:
videometa_df["snap_frame"] = videometa_df.apply(get_snap_frame, axis=1)
helmet_df = helmet_df[['game_play', 'view', 'frame', 'nfl_player_id', 'left', 'width', 'top', 'height']]

In [ ]:
print("all game play num = " ,len(target_df["game_play"].unique()))

if not DEBUG:
    train_gameplays = target_df["game_play"].unique()[:50]
    valid_gameplays = target_df["game_play"].unique()[-10:]
else:
    train_gameplays = target_df["game_play"].unique()[:2]
    valid_gameplays = target_df["game_play"].unique()[-1:]

print(len(train_gameplays), len(valid_gameplays))
train_videos = [[gameplay,"Endzone"] for gameplay in train_gameplays]
valid_videos = [[gameplay,"Endzone"] for gameplay in valid_gameplays]

# Dataset

In [ ]:
class NFLDataset(Dataset):
    def __init__(self, game_play, view, target_df, helmet_df, meta_df, transform=None):
        self.target_df = target_df
        self.helmet_df = helmet_df
        self.meta_df = meta_df
        self.transform = transform
        self.game_play = game_play
        video_file = game_play + "_" + view + ".mp4"
        self.view = view
        self.video_path = os.path.join(CFG.BASE_DIR, "train", video_file)
        self.cam = cv2.VideoCapture(self.video_path)

    def __len__(self):
        return len(self.target_df)

    def __getitem__(self, idx):
        target_info = self.target_df.iloc[idx]
        target = target_info.contact
        game_play1, game_play2, step, player1, player2 = target_info.contact_id.split("_")
        # game_play = game_play1 + "_" + game_play2
        # view = "Endzone"
        meta_info = self.meta_df.query('game_play==@self.game_play and view==@self.view')
        snap_frame = int(meta_info.snap_frame)
        read_frame = snap_frame + int(step) 
        # print(read_frame)
        self.cam.set(cv2.CAP_PROP_POS_FRAMES, int(read_frame))
        ret, img = self.cam.read()
        if ret:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = np.clip(img, 0, 255).astype(np.uint8)
            player1 = int(player1)
            helmet_info = self.helmet_df.query('game_play==@self.game_play and view==@self.view')
            # player1_helmet = helmet_info.query('nfl_player_id==@player1 and frame==@read_frame')
            player1_helmet = search_helmet(helmet_info, player1, read_frame)
            if player2 == "G":
                if len(player1_helmet) > 0:
                    # print("only player1 p2=G")
                    helmet_pos = [player1_helmet.left.values[0], player1_helmet.top.values[0],
                                  player1_helmet.width.values[0], player1_helmet.height.values[0]]
                    crop_area = get_1player_croparea(helmet_pos)
                    mask = make_player_mask(img, helmet_pos)
                    img = mask_blend(img, mask)
                    img = img[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2], :]
                else:
                    # print("no player p2=G")
                    pass
            else:
                player2 = int(player2)
                # player2_helmet = helmet_info.query('nfl_player_id==@player2 and frame==@read_frame')
                player2_helmet = search_helmet(helmet_info, player2, read_frame)
                if len(player2_helmet) == 0 and len(player1_helmet)==0:
                    # print("no player len0 p2 not G")
                    pass
                elif len(player1_helmet) > 0 and len(player2_helmet) == 0:
                    # print("only player1 p2notG")
                    helmet_pos = [player1_helmet.left.values[0], player1_helmet.top.values[0],
                                  player1_helmet.width.values[0], player1_helmet.height.values[0]]
                    # img = make_player_mask(img, helmet_pos)
                    mask = make_player_mask(img, helmet_pos)
                    img = mask_blend(img, mask)
                    crop_area = get_1player_croparea(helmet_pos)
                    img = img[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2], :]                    
                elif len(player2_helmet) > 0 and len(player1_helmet) == 0:
                    # print("only player2")
                    helmet_pos = [player2_helmet.left.values[0], player2_helmet.top.values[0],
                                  player2_helmet.width.values[0], player2_helmet.height.values[0]]
                    # img = make_player_mask(img, helmet_pos)
                    mask = make_player_mask(img, helmet_pos)
                    img = mask_blend(img, mask)
                    crop_area = get_1player_croparea(helmet_pos)
                    img = img[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2], :]
                else:
                    # print("two players")
                    helmet1_pos = [player1_helmet.left.values[0], player1_helmet.top.values[0],
                                  player1_helmet.width.values[0], player1_helmet.height.values[0]]
                    helmet2_pos = [player2_helmet.left.values[0], player2_helmet.top.values[0],
                                  player2_helmet.width.values[0], player2_helmet.height.values[0]]
                    mask1 = make_player_mask(img, helmet1_pos)
                    mask2 = make_player_mask(img, helmet2_pos)
                    mask = mask1 + mask2
                    img = mask_blend(img, mask)
                    crop_area = get_2player_croparea(helmet1_pos, helmet2_pos)
                    img = img[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2], :]

            img = cv2.resize(img, dsize=CFG.img_size)
            img = img / 255. # convert to 0-1
            img = np.transpose(img, (2, 0, 1)).astype(np.float32)
            img = torch.tensor(img, dtype=torch.float)
            target = torch.tensor(target, dtype=torch.float)
            return img, target
        else:
            img = np.zeros(CFG.img_size)
            img = np.transpose(img, (2, 0, 1)).astype(np.float32)
            img = torch.tensor(img, dtype=torch.float)
            target = torch.tensor(target, dtype=torch.float)
            return img, target

# Model

In [ ]:
# without meta
class NFLNet(nn.Module):
    def __init__(
        self,
        model_name = CFG.model_name,
        out_features = CFG.out_features,
        inp_channels= CFG.inp_channels,
        pretrained = CFG.pretrained
    ):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels, num_classes = out_features)
    
    def forward(self, image):
        output = self.model(image)
        return output

# Loss

In [ ]:
# criterion = nn.BCEWithLogitsLoss() #define in train loop

# train fn

In [ ]:
def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler):
    model.train()
    batch_time = AverageMeter()
    losses = AverageMeter()
    start = end = time.time()
    for batch_idx, (images, targets) in enumerate(train_loader):    
        images = images.to(device, non_blocking = True).float()
        targets = targets.to(device, non_blocking = True).float().view(-1, 1)
                
        preds = model(images)
        loss = criterion(preds, targets)
        mlflow.log_metric("train loss", loss.item(), step=batch_idx)
        losses.update(loss.item(), CFG.batch_size) 
        
        targets = targets.detach().cpu().numpy().ravel().tolist()
        preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

        loss.backward() # パラメータの勾配を計算
        
        optimizer.step() # モデル更新
        optimizer.zero_grad() # 勾配の初期化
                
        batch_time.update(time.time() - end)
        end = time.time()
        if batch_idx % CFG.print_freq == 0 or batch_idx == (len(train_loader)-1):
            print('\t Epoch: [{0}][{1}/{2}] '
                    'Elapsed {remain:s} '
                    'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                    .format(
                        epoch, batch_idx, len(train_loader), batch_time=batch_time, loss=losses,
                        remain=timeSince(start, float(batch_idx+1)/len(train_loader)),
            ))
        # if batch_idx > 2:
        #     break
        del preds, images, targets
    del batch_time, model
    gc.collect()
    torch.cuda.empty_cache()
    return losses.avg

# valid fn

In [ ]:
def valid_fn(model, criterion, valid_videos):
    model.eval()# モデルを検証モードに設定
    test_targets = []
    test_preds = []
    for game_play, view in valid_videos:
        print(f"[valid] game_play={game_play}, view={view}")
        target_game = target_df.query('game_play==@game_play')
        helmet_game = helmet_df.query('game_play==@game_play and view==@view')
        videometa_game = videometa_df.query('game_play==@game_play and view==@view')
        valid_dataset = NFLDataset(game_play, view, target_game, helmet_game, videometa_game)
            
        valid_loader = DataLoader(
            valid_dataset,
            batch_size = CFG.batch_size,
            shuffle = False,
            num_workers = CFG.num_workers,
            pin_memory = True
        )
        batch_time = AverageMeter()
        losses = AverageMeter()
        start = end = time.time()
        for batch_idx, (images, targets) in enumerate(valid_loader):
            images = images.to(device, non_blocking = True).float()
            targets = targets.to(device, non_blocking = True).float().view(-1, 1)
            with torch.no_grad():
                preds = model(images)
                loss = criterion(preds, targets)
                mlflow.log_metric("valid loss", loss.item(), step=batch_idx)
                losses.update(loss.item(), CFG.batch_size)
                batch_time.update(time.time() - end)
                
                targets = targets.detach().cpu().numpy().ravel().tolist()
                preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()
                # targets = (np.array(targets) > 0.5).astype(int)
                # preds = (np.array(preds) > 0.5).astype(int)
            
                test_preds.extend(preds)
                test_targets.extend(targets)
                # score = matthews_corrcoef(preds, targets)
                if batch_idx % CFG.print_freq == 0 or batch_idx == (len(valid_loader)-1):
                    print('\t EVAL: [{0}/{1}] '
                        'Elapsed {remain:s} '
                        'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                        .format(
                            batch_idx, len(valid_loader), batch_time=batch_time, loss=losses,
                            remain=timeSince(start, float(batch_idx+1)/len(valid_loader)),
                        ))
            # if batch_idx > 2:
            #     break
        
            del preds, images, targets
        
        test_preds = np.array(test_preds)
        test_targets = np.array(test_targets)
        del valid_loader, valid_dataset
        gc.collect()
        torch.cuda.empty_cache()
    return test_targets, test_preds, losses.avg

# Train Loop

In [ ]:
def training_loop(train_videos, valid_videos):
    oof_df = pd.DataFrame()
    """
    instantiate model, cost function and optimizer
    """
    model = NFLNet()
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    # norm_bias_params, non_norm_bias_params = divice_norm_bias(model)
    optimizer = AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
    
    """ 
    Set dataset
    """
    # train_dataset = ""
    best_score = -np.inf
    start_time = end = time.time()
    for epoch in range(1, CFG.n_epoch + 1):
        print(f'=== epoch: {epoch}: training ===')
        for data_idx, (game_play, view) in enumerate(train_videos):
            print(f"[train] game_play={game_play}, view={view}")
            target_game = target_df.query('game_play==@game_play')
            helmet_game = helmet_df.query('game_play==@game_play and view==@view')
            videometa_game = videometa_df.query('game_play==@game_play and view==@view')
            train_dataset = NFLDataset(game_play, view, target_game, helmet_game, videometa_game)

            train_loader = DataLoader(
                train_dataset,
                batch_size = CFG.batch_size,
                shuffle = False,
                num_workers = CFG.num_workers,
                pin_memory = True
            )
            """
            train / valid loop
            """
            train_loss_avg = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler)
            mlflow.log_metric("train loss avg", train_loss_avg, step=epoch+data_idx)                 
        valid_targets, valid_preds, valid_loss_avg = valid_fn(model, criterion, valid_videos)
        mlflow.log_metric("valid loss avg", valid_loss_avg, step=epoch)
        # valid_score = matthews_corrcoef(valid_preds, valid_targets)
        
        valid_score = 0
        valid_threshold = 0
        for idx in range(1, 10, 1):
            thr = idx*0.1
            valid_targets = (np.array(valid_targets) > thr).astype(np.int32)
            valid_preds = (np.array(valid_preds) > thr).astype(np.int32)
            score_tmp = matthews_corrcoef(valid_targets, valid_preds)
            mlflow.log_metric("valid score tmp", score_tmp, step=epoch+idx)
            if score_tmp > valid_score:
                valid_score = score_tmp 
                valid_threshold = thr           
        
        mlflow.log_metric("valid score", valid_score, step=epoch)
        mlflow.log_metric("valid threshold", valid_threshold, step=epoch)
        elapsed = time.time() - start_time
        print(f'epoch:{epoch}, avg train loss:{train_loss_avg:.4f}, avg valid loss:{valid_loss_avg:.4f}, score:{valid_score:.4f}(th={valid_threshold}) ::: time:{elapsed:.2f}s')
        # scheduler.step(valid_score)
        scheduler.step()
        # validationスコアがbestを更新したらモデルを保存する
        if valid_score > best_score:
            best_score = valid_score
            model_name = CFG.model_name
            # torch.save(model.state_dict(), f'{CFG.model_dir}/{model_name}_fold{i_fold}.pth')
            torch.save(model.state_dict(), f'{CFG.model_dir}/{model_name}.pth')
            print(f'Epoch {epoch} - Save Best Score: {best_score:.4f}. Model is saved.')
            _oof_df = pd.DataFrame({
                "pred" : valid_preds,
                "target" : valid_targets,
            })
            # _oof_df = pd.DataFrame(data={'Id': valid_ids, 'pred':preds, 'fold': i_fold, 'Pawpularity':valid_targets}, index=valid_idx)

        del train_loader, train_dataset
        gc.collect()
        
        torch.cuda.empty_cache()
        oof_df = pd.concat([oof_df, _oof_df], axis = 1)
        return oof_df

# RUN EXP

In [ ]:
experiment = mlflow.get_experiment_by_name(CFG.MLFLOW_CATEGORY)
if experiment is None:  # 当該Experiment存在しないとき、新たに作成
    experiment_id = mlflow.create_experiment(
                            name=CFG.MLFLOW_CATEGORY)
else: # 当該Experiment存在するとき、IDを取得
    experiment_id = experiment.experiment_id

# mlflow.pytorch.autolog(log_every_n_epoch=1)
with mlflow.start_run(experiment_id=experiment_id) as run:
    oof_df = training_loop(train_videos, valid_videos)

In [ ]:
output_dir = os.path.join(os.path.dirname(CFG.BASE_DIR), "output")
filename = os.path.join(output_dir, f"oof_{CFG.EXP_NAME}.csv")
num = 0
while os.path.exists(filename):
    print(filename)
    filename = os.path.join(output_dir, f"oof_{CFG.EXP_NAME}_{num}.csv")
    num += 1
display(oof_df)
display(oof_df["target"].value_counts())
oof_df.to_csv(filename, index=False)

In [ ]:
display(oof_df)
valid_df = pd.DataFrame()
for game_play, view in valid_videos:
    target_game = target_df.query('game_play==@game_play')
    valid_tmp = target_game[["game_play", "contact"]]
    valid_df = pd.concat([valid_df, valid_tmp])
print(len(valid_df), len(oof_df))